# Szegedy’s quantum walk with queries (Santos, 2016)

Implementation of the algorithm described in [this paper](https://link.springer.com/article/10.1007/s11128-016-1427-4).

## Import libraries

In [46]:
import pennylane as qml
import pennylane.numpy as np
from scipy.linalg import block_diag

In [118]:
np.set_printoptions(threshold=np.inf)
qml.__version__

'0.41.1'

## 2 Szegedy's Quatum Walk

In [127]:
N = 4 # dimension of the graph
n = int(np.ceil(np.log2(N))) # dimension of the associated Hilbert space
print(f"N = {N}")
print(f"n = {n}")

N = 4
n = 2


### Stochastic matrices

![Alt text](complete_graph_m1.PNG "Complete graph with m=5, n=1")

#### $P'$

In [128]:
m = 2 # marked state
absorbing = False

# transition matrix
P = block_diag(1/(N-1) * (np.ones(N) - np.eye(N)), np.eye(2**n-N))
# absorbing RW
if (absorbing):
    for i in range(N):
        P[m-1, i] = 0
    P[m-1, m-1] = 1

print(P)
print(P.shape)
# print(P*P.T)

[[0.         0.33333333 0.33333333 0.33333333]
 [0.33333333 0.         0.33333333 0.33333333]
 [0.33333333 0.33333333 0.         0.33333333]
 [0.33333333 0.33333333 0.33333333 0.        ]]
(4, 4)


#### $R_A$

In [ ]:
# initial state of the quantum walk
RA = np.zeros((2**(2*n), 2**(2*n)))

for x in range(2**n):
    ket_x = np.zeros((2**n, 1))
    ket_x[x] = 1
    for y in range(2**n):
        ket_y = np.zeros((2**n, 1))
        ket_y[y] = np.sqrt(P[x][y])
        phi = np.kron(ket_x, ket_y) # x⊗y
    # print(f"x={x}")
    # print(phi.T)   
    RA += np.dot(phi, phi.T)    
RA = 2 * RA - np.eye(2**(2*n))
print(RA.shape)
print(RA)
print(r'Is $R_A$ unitary?', np.all(np.equal(RA*RA.T, np.eye(2**(2*n)))))

(64, 64)
[[-1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

#### $R_B$

In [129]:
# initial state of the quantum walk
RB = np.zeros((2**(2*n), 2**(2*n)))

for y in range(2**n):
    psi = np.zeros((2**(2*n), 1))
    ket_y = np.zeros((2**n, 1))
    ket_y[y] = 1
    for x in range(2**n):
        ket_x = np.zeros((2**n, 1))
        ket_x[x] = np.sqrt(P[y][x])
        psi += np.kron(ket_x, ket_y) # y⊗x
    # psi /= np.sqrt(2**n)
    RB += np.dot(psi, psi.T)    
RB = 2 * RB - np.eye(2**(2*n))
print(RB.shape)
print(RB)
print(r'Is $R_B$ unitary?', np.all(np.equal(RB*RB.T, np.eye(2**(2*n)))))

(16, 16)
[[-1.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.         -0.33333333  0.          0.          0.          0.
   0.          0.          0.          0.66666667  0.          0.
   0.          0.66666667  0.          0.        ]
 [ 0.          0.         -0.33333333  0.          0.          0.
   0.66666667  0.          0.          0.          0.          0.
   0.          0.          0.66666667  0.        ]
 [ 0.          0.          0.         -0.33333333  0.          0.
   0.          0.66666667  0.          0.          0.          0.66666667
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.         -0.33333333  0.
   0.          0.          0.66666667  0.          0.          0.
   0.66666667  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -1.
  